In [32]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [33]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-02-08 21:26:46--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-02-08 21:26:46 (5.92 MB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()


# EXTRACT

In [35]:
# Read in the data from S3 Buckets from Amazon
from pyspark import SparkFiles
# Load in Outdoor data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy-mm-dd")
df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-01-31 00:08:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

# TRANSFORM

## Counting Records: Drop duplicates and incomplete rows

In [36]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

145431
145425
145425


## Examine the schema

In [37]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



review_id_table create

In [18]:
## Creater review_id_table DataFrame
review_id_table_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_table_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R1EB82XGHB4AXH|   40847927|B003YCQF3C|     452532551|2015-01-31 00:08:00|
|R10DEIO121647A|   47628015|B00C7B0G9A|     872270797|2015-01-30 00:08:00|
|R35QUODBLL4EZY|   38321201|B00B7S3KYM|     186820002|2015-01-30 00:08:00|
|R1FWYLPPOALS93|   24626672|B00F4CEHNK|     341969535|2015-01-23 00:08:00|
| R8595SHEBU8G7|   15954061|B010KYDNDG|     835376637|2015-01-22 00:08:00|
|R1ZNBPJ7NVOUZU|   17216137|B004RMK4BC|     384246568|2015-01-21 00:08:00|
|R1T6XC74YL1LDH|    3567827|B00KN7B1VM|     579541505|2015-01-20 00:08:00|
|R17A3O274JIQL8|   14980848|B0043EWK5W|     712579588|2015-01-20 00:08:00|
| RZTT5EIPOZKIW|   20470522|B003BVIXWW|      20478250|2015-01-18 00:08:00|
|R319RKHRPKYQJU|    7476163|B004RMK4BC|     384246568|2015-01-14 00:08:00|
| R9UW1ROSUUIOL|    20522

In [38]:
# Print schema
review_id_table_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



Products table

In [39]:
# Create product dataframe to match products table(only unique values)
products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001TK3VTC|Cubis Gold 2 [Dow...|
|B002MUB0TG|Little Shop of Tr...|
|B00DRO824C|X2 The Threat [On...|
|B00CMDKNHI| 100% Hidden Objects|
|B00CMSCTA2|Age of Empires II...|
|B00GDHD3LW|1 Month Subscript...|
|B00OM1VVJM|            NBa 2K15|
|B005XOJ6NK|   Sonic Generations|
|B00GGUHGGY|Call of Duty Blac...|
|B00A9XQ890|Grand Theft Auto:...|
|B003YMMGEE|Sniper: Ghost War...|
|B00405RA04|Rome: Total War -...|
|B0083KX9IO|Stronghold Kingdo...|
|B00M0W1YZQ|Chronicles of Wit...|
|B00895ARJC|  Syberia [Download]|
|B0084FHQ1O|The Stronghold Co...|
|B00KK9PR90|Freddi Fish 3: Th...|
|B001KC01OW|Wonderlines [Down...|
|B008TTPTOQ|Battlefield Bad C...|
|B00DGZDGUM|Nascar The Game 2...|
+----------+--------------------+
only showing top 20 rows



Customers table

In [40]:
# Create customers dataframe to match customers table
from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   21925455|            95|
|   40955995|            92|
|   23564949|            83|
|   52520442|            73|
|   14279279|            72|
|   51520185|            65|
|   20751317|            54|
|   25575022|            49|
|   50529050|            49|
|   11584162|            49|
|   36527466|            48|
|   39942708|            48|
|   31012456|            46|
|   52919259|            45|
|   19371429|            43|
|   46544620|            43|
|   53080901|            42|
|   26151231|            42|
|   45688707|            40|
|   44751489|            39|
+-----------+--------------+
only showing top 20 rows



Vine table

In [41]:
# Create vine_table DataFrame

# Load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

vine_table_df.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1EB82XGHB4AXH|          4|            0|          0|   N|
|R10DEIO121647A|          1|            1|          3|   N|
|R35QUODBLL4EZY|          4|            0|          0|   N|
|R1FWYLPPOALS93|          5|            0|          0|   N|
| R8595SHEBU8G7|          5|            0|          0|   N|
|R1ZNBPJ7NVOUZU|          5|            0|          0|   N|
|R1T6XC74YL1LDH|          4|            0|          0|   N|
|R17A3O274JIQL8|          1|            0|          0|   N|
| RZTT5EIPOZKIW|          1|            0|          0|   N|
|R319RKHRPKYQJU|          5|            0|          0|   N|
| R9UW1ROSUUIOL|          4|            1|          1|   N|
|R2WN78GAUPS5MG|          5|            0|          0|   N|
|R1JOIS4WX1829R|          5|            0|          0|   N|
|R304ZAYNDIKXY2|          5|            

In [ ]:
# DataFrame to RDS
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<edit insert connection here>:5432/<edit db name>"
config = {"user":"root",
          "password": "<insert password>",
          "driver":"org.postgresql.Driver"}

Load DataFrame to RDS

In [43]:
# Write DataFrame to RDS
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


In [44]:
# Write DataFrame to products table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)


In [45]:
# Write DataFrame to customers table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)


In [46]:
# Write DataFrame to vine_table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)
